# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

W. Brandner  ->  W. Brandner  |  ['W. Brandner']
G. Chauvin  ->  G. Chauvin  |  ['G. Chauvin']
P. Garcia  ->  A. P. Garcia  |  ['P. Garcia']
T. Henning  ->  T. Henning  |  ['T. Henning']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
P. Mollière  ->  P. Mollière  |  ['P. Mollière']
S. Torniamenti  ->  S. Torniamenti  |  ['S. Torniamenti']
K. Lee  ->  K. Lee  |  ['K. Lee']


R. Burn  ->  R. Burn  |  ['R. Burn']
T. Suhasaria  ->  T. Suhasaria  |  ['T. Suhasaria']
V. Leuschner  ->  V. Leuschner  |  ['V. Leuschner']
C. Jaeger  ->  C. Jaeger  |  ['C. Jaeger']
C. Gieser  ->  C. Gieser  |  ['C. Gieser']
T. Henning  ->  T. Henning  |  ['T. Henning']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
Arxiv has 87 new papers today
          8 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/8 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2507.00117


extracting tarball to tmp_2507.00117...

 done.


Found 88 bibliographic references in tmp_2507.00117/aanda.bbl.
Retrieving document from  https://arxiv.org/e-print/2507.00137
extracting tarball to tmp_2507.00137...

...

Found 107 bibliographic references in tmp_2507.00933/Research_report688.bbl.
Error retrieving bib data for betremieux14: 'betremieux14'
Error retrieving bib data for LY2023b: 'ly2023b'
Error retrieving bib data for zahnle17: 'zahnle17'
Retrieving document from  https://arxiv.org/e-print/2507.00954


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem
 item = \bibitem{dressing15}CD {Dressing}, D {Charbonneau}, {The Occurrence of Potentially Habitable  Planets Orbiting M Dwarfs Estimated from the Full Kepler Dataset and an  Empirical Measurement of the Detection Sensitivity}.\newblock {\em\protect\JournalTitle{\apj}} \textbf{807}, 45 (2015).
 regex = 
        \\bibitem(\[[^\[\]]*?\]){(?P<bibkey>[a-zA-Z0-9\-\+\.\S]+?)}(?P<authors>|([\D]*?))(?P<year>[12][0-9]{3})(?P<rest>.*)
        
  warnings.warn(str(e))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem
 item = \bibitem{petigura13}EA {Petigura}, GW {Marcy}, AW {Howard}, {A Plateau in the Planet Population  below Twice the Size of Earth}.\newblock {\em\protect\JournalTitle{Astrophys.~J.~}} \textbf{770}, 69 (2013).
 regex = 
        \\bibitem(\[[^\[\]]*?\]){(?P

extracting tarball to tmp_2507.00954...

 done.


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2507.00954/submission2_nsr.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'supplementary_info.tex' from 'tmp_2507.00954/supplementary_info.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.00117-b31b1b.svg)](https://arxiv.org/abs/2507.00117) | **Orbit and atmosphere of HIP 99770 b through the eyes of VLTI/GRAVITY**  |
|| T. O. Winterhalder, et al. -- incl., <mark>W. Brandner</mark>, <mark>G. Chauvin</mark>, <mark>P. Garcia</mark>, <mark>T. Henning</mark>, <mark>L. Kreidberg</mark>, <mark>P. Mollière</mark> |
|*Appeared on*| *2025-07-02*|
|*Comments*| *Accepted for publication in Astronomy and Astrophysics*|
|**Abstract**|            Context: Inferring the likely formation channel of giant exoplanets and brown dwarf companions from orbital and atmospheric observables remains a formidable challenge. Further and more precise directly measured dynamical masses of these companions are required to inform and gauge formation, evolutionary, and atmospheric models. We present an updated study of HIP 99770 b based on observations conducted with VLTI/GRAVITY. Aims: Combining the new data with previous observations from the literature, we characterise HIP 99770 b to better constrain its orbit, dynamical mass, and atmospheric properties, as well as to shed light on its likely formation channel. Methods: We ran a renewed orbit fit to further constrain the dynamical mass of the companion and the orbit solution. We also analysed the GRAVITY K-band spectrum, placing it into context with literature data, and extracting magnitude, age, spectral type, bulk properties and atmospheric characteristics of HIP 99770 b. Results: We detected the companion at a radial separation of $417\,\mathrm{mas}$ from its host. The new orbit fit yields a dynamical mass of $17_{-5}^{+6}\,\mathrm{M}_\mathrm{Jup}$ and an eccentricity of $0.31_{-0.12}^{+0.06}$. We also find that additional relative astrometry epochs in the future will not enable further constraints on the dynamical mass due to the dominating relative uncertainty on the Hipparcos-Gaia proper motion anomaly. The publication of Gaia DR4 will likely ease this predicament. We find that the companion is consistent with spectral type L8 and exhibits a potential metal enrichment in its atmosphere. Conclusions: These results do not yet allow for a definite inference of the companion's formation channel. Nevertheless, the new constraints on its bulk properties and the additional GRAVITY spectrum presented here will aid future efforts to determine the formation history of HIP 99770 b.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.00137-b31b1b.svg)](https://arxiv.org/abs/2507.00137) | **Metal-poor single Wolf-Rayet stars: the interplay of optically thick winds and rotation**  |
|| L. Boco, et al. -- incl., <mark>S. Torniamenti</mark> |
|*Appeared on*| *2025-07-02*|
|*Comments*| *18 pages, 17 Figures. Submitted to A&A. Comments from the community are welcome*|
|**Abstract**|            The Small Magellanic Cloud (SMC) hosts 12 known Wolf-Rayet stars (WRs), seven of which apparently single. Their formation is a challenge for current stellar evolution models, because line-driven winds are generally assumed to be quenched at a metallicity Z<0.004. Here, we present a set of MESA models of single stars with zero-age main sequence masses 20-80 M_sun, considering different initial rotation speeds (Omega=0-0.7 Omega_c), metallicities (Z=0.002-0.0045), and wind mass-loss models (optically thin and thick winds). We show that if we account for optically thick winds, as described by Sabhahit+2023, fast rotating (Omega=0.6 Omega_c) single metal-poor O-type stars (with M>20 M_sun) shed their envelope and become WR stars even at the low metallicity of the SMC. The luminosity, effective temperature, evolutionary timescale, surface abundance, and rotational velocity of our simulated WR stars are compatible to the WRs observed in the SMC. We speculate that this scenario can also alleviate the excess of giant stars across the Humphreys-Davidson limit. Our results have key implications for black hole masses, (pair instability) supernova explosions, and other observable signatures.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.00765-b31b1b.svg)](https://arxiv.org/abs/2507.00765) | **Sub-Neptunes Are Drier Than They Seem: Rethinking the Origins of Water-Rich Worlds**  |
|| A. Werlen, et al. -- incl., <mark>R. Burn</mark> |
|*Appeared on*| *2025-07-02*|
|*Comments*| *Submitted to ApJL*|
|**Abstract**|            Recent claims of biosignature gases in sub-Neptune atmospheres have renewed interest in water-rich sub-Neptunes with surface oceans, often referred to as Hycean planets. These planets are hypothesized to form beyond the snow line, accreting large amounts of H$_2$O >10 wt%) before migrating inward. However, current interior models often neglect chemical equilibration between primordial atmospheres and molten interiors. Here, we compute global chemical equilibrium states for a synthetic population of sub-Neptunes with magma oceans. Although many initially accrete 5-30 wt% water, interior-atmosphere interactions destroy most of it, reducing final H$_2$O mass fractions to below 1.5 wt%. As a result, none meet the threshold for Hycean planets. Despite that, we find H$_2$O-dominated atmospheres exclusively on planets that accreted the least ice. These planets form inside the snow line, are depleted in carbon and hydrogen, and develop small envelopes with envelope mass fractions below 1%, dominated by endogenic water. In contrast, planets formed beyond the snow line accrete more volatiles, but their water is largely converted to H$_2$ gas or sequestered into the interior, resulting in low atmospheric H$_2$O mass fractions. Most H$_2$O-rich envelopes are also fully miscible with H$_2$, making a separate water layer unlikely. Our results topple the conventional link between ice accretion and water-rich atmospheres, showing instead that H$_2$O-dominated envelopes emerge through chemical equilibration in hydrogen-poor planets formed inside the snow line.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.00836-b31b1b.svg)](https://arxiv.org/abs/2507.00836) | **CO2 infrared spectra on silicate dust grain analogs: Implications for JWST observations**  |
|| <mark>T. Suhasaria</mark>, <mark>V. Leuschner</mark>, <mark>C. Jaeger</mark>, <mark>C. Gieser</mark>, <mark>T. Henning</mark> |
|*Appeared on*| *2025-07-02*|
|*Comments*| *13 pages, 6 figures, 2 tables*|
|**Abstract**|            Carbon dioxide is one of the three most abundant species within the ice mantles around dust grains inside molecular clouds. Since a substantial amount of interstellar grains is made of siliceous materials, we have studied the infrared profile of CO2 deposited on top of a bare and ice-coated amorphous silicate (MgFeSiO4) film using reflection absorption infrared spectroscopy (RAIRS). In contrast to a metal surface, the CO2 IR profile shows a relaxation of the metal surface selection rule in the presence of the bare MgFeSiO4 dust grain analog, which brings the IR profile closer to the observational spectra while maintaining the sensitivity of RAIRS. Experiments with the underlying CO and CH4 ices show that their presence facilitates structural changes toward crystalline ice for the deposited CO2 at much lower temperatures than on the polar ice layers. Warming-up experiments of CO2 showed that it tends to stay on the silicate surface for much longer than on the gold surface without the silicate layer. We noticed for the first time a split in the 13CO2 IR feature on the pure or ice-covered silicate grain as a marker for the onset of diffusion. The laboratory 13CO2 profile then closely resembles recent JWST observations of this feature around young and embedded protostars, suggesting that it can be linked to the observed feature.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.00933-b31b1b.svg)](https://arxiv.org/abs/2507.00933) | **A first look at rocky exoplanets with JWST**  |
|| <mark>L. Kreidberg</mark>, K. B. Stevenson |
|*Appeared on*| *2025-07-02*|
|*Comments*| *Accepted review for PNAS Special Feature "Exoplanets in the JWST era"*|
|**Abstract**|            Rocky exoplanet characterization has been a top priority for early James Webb Space Telescope (JWST) science operations. Several milestones have been achieved, including the most precise rocky planet transmission spectra measured to date, and the first detection of thermal emission for rocky worlds below 800 Kelvin. Despite these advances, no atmospheres have been definitively detected. Several transmission spectra show tentative evidence for molecular absorption features, but these hints are marginally significant and the spectra may be affected by stellar contamination. Features from many plausible atmospheres, including those dominated by oxygen, nitrogen, and carbon dioxide, are below the current noise level. Meanwhile, the emerging picture from thermal emission spectra is that the planets have hot daysides, consistent with either a bare rock composition or low surface pressure atmospheres (< 10 bar). Higher surface pressures and high carbon dioxide abundances are generally ruled out, assuming cloud-free atmosphere models. The absence of strong CO$_2$ features hints at a limited initial volatile inventory or rapid atmospheric escape during the planets' early lifetimes. Taken together, these results motivate a push towards higher precision data, as well as observations of cooler planets that may be more likely to retain atmospheres. As a goal for future transmission spectroscopy, we suggest a "five scale height challenge," to achieve the precision necessary to detect CO$_2$ features in nitrogen-rich atmospheres. Detecting rocky planet atmospheres remains challenging, but with JWST's excellent performance and a continuing investment of telescope time, we are optimistic these uncharted atmospheres will be detected in coming years.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.00610-b31b1b.svg)](https://arxiv.org/abs/2507.00610) | **The radiative subpulse modulation and spectral features of PSR B1929$+$10 with the whole pulse phase emission**  |
|| Z. Wang, et al. -- incl., <mark>K. Lee</mark> |
|*Appeared on*| *2025-07-02*|
|*Comments*| *15 pages, 9 figures, accepted for publication in Astronomy & Astrophysics*|
|**Abstract**|            In this work, we observe the nearby pulsar, PSR B1929$+$10, using the Five-hundred-meter Aperture Spherical radio Telescope (FAST). We find, for the first time, two new emission components with an extremely weak observed flux density of about $10^{-4}$ of the magnitude of the peak radio emission of PSR B1929$+$10. Our results show that the intrinsic radio emission of PSR B1929$+$10 covers the $360^{\circ}$ of longitude, demonstrating that this pulsar is a whole $360^{\circ}$ of longitude emission pulsar. We find at least 15 components of pulse emission in the average pulse profile. Additionally, we identify 5 modes of subpulse modulation in different emission regions, which differ from the pulse components. Moreover, the narrowband emission feature and the frequent jumps in the observed linear polarization position angle (PPA) are also detected in the single pulse of this pulsar. To understand the magnetosphere of this pulsar, we analyze the observed PPA variations across the whole $360^{\circ}$ of longitude and fit them using the classical rotating vector model (RVM). For the best-fit model, the inclination angle,$\alpha$, and the impact angle, $\beta$, of this pulsar are $55^{\circ}.56$ and $53^{\circ}.47$, respectively. Using the rotating magnetosphere approximation of the magnetic dipole field, we investigate the three-dimensional pulsar magnetosphere and the sparking pattern on the polar cap surface. Our analysis indicates that the extremely narrow zone of the polar cap, which is associated with a high-altitude magnetospheric region, is responsible for the weak emission window. This pulsar has extremely high-altitude magnetospheric radio emissions.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.00873-b31b1b.svg)](https://arxiv.org/abs/2507.00873) | **A period-increasing oscillation signal in a long gamma-ray burst**  |
|| T.-C. Zheng, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-07-02*|
|*Comments*| *14 pages, 12 figures, submitted to ApJ*|
|**Abstract**|            Gamma-ray bursts (GRBs), the brightest electromagnetic bursts in the universe, are believed to originate from ultra-relativistic jets launched by the rapidly rotating central engine, either a disk-surrounded newly formed black hole (BH) or a magnetar. Such a central engine potentially possesses rapidly evolving physical characteristics, as it is just born. The caught of time-increasing frequency in the gravitational wave signals is the evidence of upon opinion. Here we report a possible oscillatory signal identified in GRB 131122B with periods increasing from 1.27 seconds to 4.02 seconds in a time interval of 16.75 seconds. Such a peculiar oscillation signal has not been identified in GRBs before and its periodic evolution could also be the quickest one found in the electromagnetic radiation window of astrophysics. The precession of a misaligned accretion disk caused by the tidal disruption of a star by an intermediate-mass BH may be responsible for this signal. This finding could open a new window to reveal the nature of the hiding central engine of GRBs.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.00954-b31b1b.svg)](https://arxiv.org/abs/2507.00954) | **Inverse Velocity Dispersion of Solar Energetic Protons Observed by Solar Orbiter and Its Shock Acceleration Explanation**  |
|| Y. Li, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-07-02*|
|*Comments*| **|
|**Abstract**|            The particle acceleration and transport process during solar eruptions is one of the critical and long-standing problems in space plasma physics. Through decades of research, it is well accepted that particles with higher energies released during a solar eruption arrive at observers earlier than the particles with lower energies, forming a well-known structure in the dynamic energy spectrum called particle velocity dispersion (VD), as frequently observed by space missions. However, this picture is challenged by new observations from NASA's Parker Solar Probe and ESA's Solar Orbiter which show an unexpected inverse velocity dispersion (IVD) phenomenon, where particles with higher-energies arrive later at the observer. Facing on the challenge, we here report the recent discovery of such IVD structures with 10 solar energetic proton events observed by Solar Orbiter, and then analyze the mechanisms causing this unusual phenomenon. We suggest that shock diffusive acceleration, with respect to magnetic reconnection, is probably a dominant mechanism to accelerate protons to tens of MeV in such events where particles need longer time to reach higher energies. And we determine, innovatively, the physical conditions and time scales during the actual shock acceleration process that cannot be observed directly.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2507.00117/./figures/posteriors.png', 'tmp_2507.00117/./figures/age_plot.png', 'tmp_2507.00117/./figures/cmd.png']
copying  tmp_2507.00117/./figures/posteriors.png to _build/html/
copying  tmp_2507.00117/./figures/age_plot.png to _build/html/
copying  tmp_2507.00117/./figures/cmd.png to _build/html/
exported in  _build/html/2507.00117.md
    + _build/html/tmp_2507.00117/./figures/posteriors.png
    + _build/html/tmp_2507.00117/./figures/age_plot.png
    + _build/html/tmp_2507.00117/./figures/cmd.png
found figures ['tmp_2507.00137/./Plot/new_winds2.png', 'tmp_2507.00137/./Plot/HRD_abundances.png', 'tmp_2507.00137/./Plot/abundances_T.png', 'tmp_2507.00137/./Plot/cremn_mass.png']
copying  tmp_2507.00137/./Plot/new_winds2.png to _build/html/
copying  tmp_2507.00137/./Plot/HRD_abundances.png to _build/html/
copying  tmp_2507.00137/./Plot/abundances_T.png to _build/html/
copying  tmp_2507.00137/./Plot/cremn_mass.png to _build/html/
exported in  _build/html/2507.00137.md
 

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand{\arraystretch}{1.4}$
$\newcommand{\arraystretch}{1.4}$
$\newcommand{\fsed}{f_{\textrm{sed}}}$</div>



<div id="title">

# Orbit and atmosphere of HIP 99770 b through the eyes of VLTI/GRAVITY

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.00117-b31b1b.svg)](https://arxiv.org/abs/2507.00117)<mark>Appeared on: 2025-07-02</mark> -  _Accepted for publication in Astronomy and Astrophysics_

</div>
<div id="authors">

T. O. Winterhalder, et al. -- incl., <mark>W. Brandner</mark>, <mark>G. Chauvin</mark>, <mark>P. Garcia</mark>, <mark>T. Henning</mark>, <mark>L. Kreidberg</mark>, <mark>P. Mollière</mark>

</div>
<div id="abstract">

**Abstract:** Inferring the likely formation channel of giant exoplanets and brown dwarf companions from orbital and atmospheric observables remains a formidable challenge.Further and more precise directly measured dynamical masses of these companions are required to inform and gauge formation, evolutionary, and atmospheric models.We present an updated study of the recently discovered companion to HIP 99770 based on observations conducted with the near-infrared interferometer VLTI/GRAVITY. Through renewed orbital and spectral analyses based on the GRAVITY data, we characterise HIP 99770 b to better constrain its orbit, dynamical mass, and atmospheric properties, as well as to shed light on its likely formation channel. Upon inclusion of the new high-precision astrometry epoch, we ran an orbit fit to further constrain the dynamical mass of the companion and the orbit solution.We also analysed the GRAVITY K-band spectrum, placing it into context with literature data, and extracting magnitude, age, spectral type, bulk properties and atmospheric characteristics of HIP 99770 b. We detected the companion at a radial separation of $\SI{417}{mas}$ from its host. The new orbit fit yields a dynamical mass of $17_{-5}^{+6} \mathrm{M}_\mathrm{Jup}$ and an eccentricity of $0.31_{-0.12}^{+0.06}$ . We also find that additional relative astrometry epochs in the future will not enable further constraints on the dynamical mass due to the dominating relative uncertainty on the Hipparcos-- _Gaia_ proper motion anomaly that is used in the orbit-fitting routine. The publication of Gaia DR4 will likely ease this predicament.Based on the spectral analysis, we find that the companion is consistent with spectral type L8 and exhibits a potential metal enrichment in its atmosphere.Adopting the AMES-DUSTY model to infer its age, within its dynamical mass constraint the companion conceivably corresponds to either a younger ( $28_{-14}^{+15} \mathrm{Myr}$ ) object with a mass just below the deuterium-burning limit or an older ( $119_{-10}^{+37} \mathrm{Myr}$ ) body with a mass just above the deuterium-burning limit. These results do not yet allow for a definite inference of the companion's formation channel.Nevertheless, the new constraints on its bulk properties and the additional GRAVITY spectrum presented here will aid future efforts to determine the formation history of HIP 99770 b.

</div>

<div id="div_fig1">

<img src="tmp_2507.00117/./figures/posteriors.png" alt="Fig4" width="100%"/>

**Figure 4. -** Marginalised posterior distributions of a subset of the fitted orbital parameters. The black posteriors were sampled considering the previously available data only, while the green posteriors result from the inclusion of the new GRAVITY epoch from 31 May 2023. The median values and intervals between the 16th and 84th percentiles of the distribution are indicated by the horizontal bars. The inferred values resulting from the two runs, which were rounded to the respective significant figure, are displayed at the top of each panel. While technically not one of the parameters explored by the walkers during the sampling procedure, we also plot the period, $P$, as computed from the other posteriors via Kepler's third law. The full posterior samplings can be found in Appendix \ref{app_section_additional_orbital_fitting_plots}.
         (*figure_posteriors*)

</div>
<div id="div_fig2">

<img src="tmp_2507.00117/./figures/age_plot.png" alt="Fig7" width="100%"/>

**Figure 7. -** Isochrones from the AMES-Dusty model grid  ([Chabrier, et. al 2000](), [Allard, et. al 2001]())  in the mass-magnitude plane. The colours indicate the respective age. The solid black lines show the dynamical mass  of the companion and the \texttt{IRDIS\_B\_Ks} filter magnitude, and the dotted lines delineate the 16th and 84th percentiles. For the filter magnitude, we adopted a conservative systematic error (see text).
        The grey colour map in the background visualises the two-dimensional distribution of randomly drawn masses and magnitudes from their respective distributions. The dash-dotted line encircles \SI{68}{\percent} of all draws.
        The shoulder in the isochrones in this particular region of the mass-magnitude plane is caused by the onset of deuterium burning and the mass dependence of its time evolution.
        The panel in the bottom right corner shows the companion age posterior obtained by bootstrapping the drawn mass and magnitude pairs onto the age manifold as defined by the isochrones.
         (*figure_age_plot*)

</div>
<div id="div_fig3">

<img src="tmp_2507.00117/./figures/cmd.png" alt="Fig5" width="100%"/>

**Figure 5. -** Colour-magnitude diagram showing HIP 99770 b, indicated by the black diamond, in relation to a literature population of low-mass stars, brown dwarfs, and exoplanets (see Appendix C of  ([Bonnefoy, Perraut and Lagrange 2018]())  and references therein). All ExoGRAVITY targets (ESO ID 1104.C-0651;  ([Lacour, Wang and Nowak 2020]()) ) and the companions so far detected via the _Gaia_--GRAVITY synergy  ([Winterhalder, Lacour and Mérand 2024]())  are shown in light blue and ochre, respectively.
        Additionally, isochrones computed using the AMES-Dusty  ([Chabrier, et. al 2000](), [Allard, et. al 2001]()) , AMES-Cond  ([Allard, et. al 2001](), [Baraffe, et. al 2003]()) , and \texttt{ATMO} ([Phillips, Tremblin and Baraffe 2020]())  evolutionary models are shown for two ages: \SI{1}{Gyr}(solid lines), and \SI{100}{Myr}(dotted lines).
        The filters we used to extract these magnitudes and colours from the spectra are \texttt{Paranal}\texttt{/}\texttt{SPHERE}\texttt{.}\texttt{IRDIS\_B\_Ks}, \texttt{IRDIS\_D\_K12\_1}, and \texttt{IRDIS\_D\_K12\_2}.
         (*figure_cmd*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.00117"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\orcidicon}[1]{\href{https://orcid.org/#1}{\includegraphics[width=11pt]{Plot/ORCIDiD_icon128x128.png}}}$
$\newcommand{\orcid}[1]{\href{https://orcid.org/#1}{\protect\orcidicon{#1}}}$
$\newcommand{\msun}{{\rm M}_\odot}$
$\newcommand{\as}[1]{\textbf{\textcolor{orange}{Andreas: {#1}}}}$
$\newcommand{\astext}[1]{{\textcolor{orange}{#1}}}$
$\newcommand{\ST}[1]{\textcolor{steelblue!100}{#1_{\mathrm{ST}}}}$
$\newcommand{\micmap}[1]{\textcolor{purple}{#1}}$
$\newcommand{\lb}[2]{\textcolor{red}{#1}}$</div>



<div id="title">

# Metal-poor single Wolf-Rayet stars:\ the interplay of optically thick winds and rotation

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.00137-b31b1b.svg)](https://arxiv.org/abs/2507.00137)<mark>Appeared on: 2025-07-02</mark> -  _18 pages, 17 Figures. Submitted to A&A. Comments from the community are welcome_

</div>
<div id="authors">

L. Boco, et al. -- incl., <mark>S. Torniamenti</mark>

</div>
<div id="abstract">

**Abstract:** The Small Magellanic Cloud (SMC) hosts 12 known Wolf-Rayet stars (WRs), seven of which apparently single. Their formation is a challenge for current stellar evolution models, because line-driven winds are generally assumed to be quenched at a metallicity $Z\leq 0.004$ . Here, we present a set of mesa models of single stars with zero-age main sequence masses $20-80$ M $_\odot$ , considering different initial rotation speeds ( $\Omega=0-0.7  \Omega_c$ ), metallicities ( $Z=0.002-0.0045$ ), and wind mass-loss models (optically thin and thick winds). We show that if we account for optically thick winds, as described by [Sabhahit, et. al (2023)]() , fast rotating ( $\Omega\sim 0.6  \Omega_c$ ) single metal-poor O-type stars (with $M\gtrsim 20$ M $_\odot$ ) shed their envelope and become WR stars even at the low metallicity of the SMC. The luminosity, effective temperature, evolutionary timescale, surface abundance, and rotational velocity of our simulated WR stars are compatible to the WRs observed in the SMC. We speculate that this scenario can also alleviate the excess of giant stars across the Humphreys-Davidson limit. Our results have key implications for black hole masses, (pair instability) supernova explosions, and other observable signatures.

</div>

<div id="div_fig1">

<img src="tmp_2507.00137/./Plot/new_winds2.png" alt="Fig15" width="100%"/>

**Figure 15. -** Stellar tracks on the HR diagram for initial masses $M=25, 30,  40,  60,  80$ M$_\odot$ at metallicity $Z=0.0025$. In the left-hand panels, \citetalias{Vink2001} model with only optically thin winds is implemented. In the right-hand panels, the \citetalias{Sabhahit2023} model is enforced, with the possibility to activate optically thick winds. The upper panels show the case with no rotation, while the lower panels the $\Omega/\Omega_c=0.6$ case. Different line styles represent different wind regimes: dotted for optically thin winds, solid for optically thick winds, dashed for WR-type winds. Colors represent different burning stages, defined as the element whose burning generates most of the energy of the star. Blue is for H-burning through CNO, orange for He-burning through triple $\alpha$, red for carbon burning, and green for oxygen burning. Green circles are observations of single WRs, filled for the hottest, open for the coldest. The lower-right plot shows that the combination of optically thick winds and high initial rotation is able to self-strip stars even at SMC metallicity. The stars in the lower-right plot (\citetalias{Sabhahit2023}, $\Omega=\Omega_c=0.6$) avoid the HD limit and transition from main-sequence stars to WRs. (*fig:new_winds*)

</div>
<div id="div_fig2">

<img src="tmp_2507.00137/./Plot/HRD_abundances.png" alt="Fig5.1" width="50%"/><img src="tmp_2507.00137/./Plot/abundances_T.png" alt="Fig5.2" width="50%"/>

**Figure 5. -** The upper panel shows the classification of the star along the HR diagram: O-type (black), WNh (blue), WN (orange), WC and WO (red). The lower panel shows the
   surface abundances as a function of the effective temperature $T_{\rm eff}$. Line styles are for different elements: hydrogen (dotted line), $^{14}$N (solid line), $^{12}$C (dashed line). Different colors are for different initial masses: 20 (blue), 30 (orange), 40 (red), and 50 M$_\odot$(green). Green triangles (circles) represent hydrogen (nitrogen) abundances for the 5 observed hottest WRs  ([Hainich, Pasemann and Todt 2015]()) . The gray horizontal band covers the $\sim 5-40\%$ range; if hydrogen abundance is inside this region the star is considered a WNh. (*fig:abundances*)

</div>
<div id="div_fig3">

<img src="tmp_2507.00137/./Plot/cremn_mass.png" alt="Fig9" width="100%"/>

**Figure 9. -** Relation between $M_{\rm ZAMS}$ and final mass of the star ($M_{\rm fin}$, thin solid line), He core mass ($M_{\rm He}$, thick dotted line), CO core mass ($M_{\rm CO}$, thick dot-dashed line) and black hole mass ($M_{\rm BH}$, thick solid line), for optically thin winds (\citetalias{Vink2001}, magenta) and for the \citetalias{Sabhahit2023} model (\citetalias{Sabhahit2023}, black). Upper panel: $\Omega/\Omega_c=0$; lower panel: $\Omega/\Omega_c=0.6$. Gray shaded area: range of $M_{\rm He}$ in which we expect a pulsational pair instability supernova. (*fig:BH_masses*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.00137"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# Sub-Neptunes Are Drier Than They Seem: Rethinking the Origins of Water-Rich Worlds

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.00765-b31b1b.svg)](https://arxiv.org/abs/2507.00765)<mark>Appeared on: 2025-07-02</mark> -  _Submitted to ApJL_

</div>
<div id="authors">

A. Werlen, et al. -- incl., <mark>R. Burn</mark>

</div>
<div id="abstract">

**Abstract:** Recent claims of biosignature gases in sub-Neptune atmospheres have renewed interest in water-rich sub-Neptunes with surface oceans, often referred to as Hycean planets. These planets are hypothesized to form beyond the snow line, accreting large amounts of $\ce{H2O}$ ( $>$ 10 wt \% ) before migrating inward. However, current interior models often neglect chemical equilibration between primordial atmospheres and molten interiors. Here, we compute global chemical equilibrium states for a synthetic population of sub-Neptunes with magma oceans. Although many initially accrete 5–30 wt \% water, interior–atmosphere interactions destroy most of it, reducing final $\ce{H2O}$ mass fractions to below 1.5 wt \% . As a result, none meet the threshold for Hycean planets. Despite that, we find $\ce{H2O}$ -dominated atmospheres exclusively on planets that accreted the least ice. These planets form inside the snow line, are depleted in carbon and hydrogen, and develop small envelopes with envelope mass fractions below 1 \% , dominated by endogenic water. In contrast, planets formed beyond the snow line accrete more volatiles, but their water is largely converted to $H_2$ gas or sequestered into the interior, resulting in low atmospheric $\ce{H2O}$ mass fractions. Most $\ce{H2O}$ -rich envelopes are also fully miscible with $\ce{H2}$ , making a separate water layer unlikely. Our results topple the conventional link between ice accretion and water-rich atmospheres, showing instead that $\ce{H2O}$ -dominated envelopes emerge through chemical equilibration in hydrogen-poor planets formed inside the snow line.

</div>

<div id="div_fig1">

<img src="tmp_2507.00765/./H2O_comparison.png" alt="Fig1" width="100%"/>

**Figure 1. -** Comparison between the accreted and equilibrated water mass fractions of sub-Neptunes. The black dashed line indicates the 1:1 correlation; in the absence of chemistry, all planets would lie along this line. The grey shaded region denotes the 10–90 wt\% water mass fraction range proposed for Hycean planets by madhusudhan_habitability_2021. All planets in our sample exhibit significant water depletion following equilibration and fall well below the Hycean threshold. (*fig:H2O_comparison*)

</div>
<div id="div_fig2">

<img src="tmp_2507.00765/./H2O_vs_atm_mass.png" alt="Fig2" width="100%"/>

**Figure 2. -** Envelope $H_2$O mass fraction as a function of envelope mass fraction, using the same dataset as in Figure \ref{fig:H2O_comparison}. The colorbar indicates the molar bulk C/O ratio. Planets with low envelope mass fractions tend to retain a higher proportion of $H_2$O in the gas phase. Moreover, planets with low C/O ratios consistently show higher $H_2$O content compared to their carbon-rich counterparts. (*fig:H2O_comparison_env*)

</div>
<div id="div_fig3">

<img src="tmp_2507.00765/./orbital_distribution.png" alt="Fig5" width="100%"/>

**Figure 5. -** Envelope $H_2$O mass fraction as a function of semi-major axis. The same dataset as in Figure \ref{fig:H2O_comparison} is shown. The left panel shows planets that predominantly formed inside the water ice line; the right panel shows those that formed outside. Classification is based on the accreted $H_2$O mass fraction, with a threshold set at 5\% of the total planetary mass. The colorbar indicates the molar bulk C/O ratio. Planets formed inside the ice line are systematically depleted in carbon due to the lack of volatile ice accretion and exhibit higher envelope $H_2$O mass fractions. In contrast, planets formed beyond the ice line retain lower $H_2$O content despite higher bulk volatile abundances. Each pie chart shows the mean mass fraction of hydrogen in $H_2$(gas), H (metal), $H_2$(silicate), $H_2$O (gas), and $H_2$O (silicate), normalized to the total mean hydrogen inventory for each population. Only components contributing more than 5\% are labeled. Planets that formed beyond the ice line store most hydrogen as $H_2$(gas) and H (metal), while those that formed inside the ice line retain a larger share of hydrogen in H (metal) $H_2$(silicate) and $H_2$O (gas + silicate). (*fig:orbital_distr.*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.00765"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\arraystretch}{1.3}$</div>



<div id="title">

# $CO_2$ infrared spectra on silicate dust grain analogs: Implications for JWST observations

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.00836-b31b1b.svg)](https://arxiv.org/abs/2507.00836)<mark>Appeared on: 2025-07-02</mark> -  _13 pages, 6 figures, 2 tables_

</div>
<div id="authors">

<mark>T. Suhasaria</mark>, et al. -- incl., <mark>V. Leuschner</mark>, <mark>C. Gieser</mark>, <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** Carbon dioxide is one of the three most abundant species within the ice mantles around dust grains inside molecular clouds. Since a substantial amount of interstellar grains is made of siliceous materials, we have studied the infrared profile of $CO_2$ deposited on top of a bare and ice-coated amorphous silicate ($MgFeSiO_4$ ) film using reflection absorption infrared spectroscopy (RAIRS). In contrast to a metal surface, the $CO_2$ IR profile shows a relaxation of the metal surface selection rule in the presence of the bare $MgFeSiO_4$ dust grain analog, which brings the IR profile closer to the observational spectra while maintaining the sensitivity of RAIRS. Experiments with the underlying CO and $CH_4$ ices show that their presence facilitates structural changes toward crystalline ice for the deposited $CO_2$ at much lower temperatures than on the polar ice layers. Warming-up experiments of $CO_2$ showed that it tends to stay on the silicate surface for much longer than on the gold surface without the silicate layer. We noticed for the first time a split in the $^{13}$ $CO_2$ IR feature on the pure or ice-covered silicate grain as a marker for the onset of diffusion. The laboratory $^{13}$ $CO_2$ profile then closely resembles recent JWST observations of this feature around young and embedded protostars, suggesting that it can be linked to the observed feature.

</div>

<div id="div_fig1">

<img src="tmp_2507.00836/./Fig3.png" alt="Fig3" width="100%"/>

**Figure 3. -** RAIRS spectra of $CO_2$($\nu_3$ vibrational mode) deposited at 15 K on gold coated with a 100 nm amorphous silicate film, with or without an additional $\sim$10 ML layer of pure or mixed ice. The top column shows 1 ML $CO_2$ spectra on (a) the silicate film without additional ice layer and on pure apolar ices, (b) on pure polar ices and an ice mixture. The bottom panels (c) and (d) (*fig:(a)polar*)

</div>
<div id="div_fig2">

<img src="tmp_2507.00836/./Fig2a.png" alt="Fig2.1" width="50%"/><img src="tmp_2507.00836/./Fig2b.png" alt="Fig2.2" width="50%"/>

**Figure 2. -** (a) RAIRS spectra of $\sim$1 ML $CO_2$($\nu_3$ vibrational mode) deposited on bare gold at 15 and 50 K and on gold coated with a 100 nm amorphous silicate film at 15 K. (b) Spectra of $\sim$1 ML $CO_2$ deposited on varying thickness of porous amorphous water layers on gold at 15 K are shown alongside the spectrum on bare gold for comparison. (*fig:LOTOsplitting*)

</div>
<div id="div_fig3">

<img src="tmp_2507.00836/./Fig6.png" alt="Fig6" width="100%"/>

**Figure 6. -** Comparison of the $^{13}$$CO_2$ ice feature toward (a) low luminous and (b) high luminous protostars with laboratory spectra on different surfaces. To facilitate comparison, all laboratory spectra have been scaled by a factor of 40. Additionally, in panel (b), the $CH_3$OH:$H_2$O spectrum is vertically offset by 0.4, while in panel (a), all laboratory spectra from panel (b) are offset by 0.6. (*fig5*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.00836"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand$
$\newcommand$</div>



<div id="title">

# A first look at rocky exoplanets with JWST

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.00933-b31b1b.svg)](https://arxiv.org/abs/2507.00933)<mark>Appeared on: 2025-07-02</mark> -  _Accepted review for PNAS Special Feature "Exoplanets in the JWST era"_

</div>
<div id="authors">

<mark>L. Kreidberg</mark>, K. B. Stevenson

</div>
<div id="abstract">

**Abstract:** Rocky exoplanet characterization has been a top priority for early James Webb Space Telescope (JWST) science operations.  Several milestones have been achieved, including the most precise rocky planet transmission spectra measured to date, and the first detection of thermal emission for rocky worlds below 800 Kelvin. Despite these advances, no atmospheres have been definitively detected. Several transmission spectra show tentative evidence for molecular absorption features, but these hints are marginally significant and the spectra may be affected by stellar contamination. Features from many plausible atmospheres, including those dominated by oxygen, nitrogen, and carbon dioxide, are below the current noise level.  Meanwhile, the emerging picture from thermal emission spectra is that the planets have hot daysides, consistent with either a bare rock composition or low surface pressure atmospheres ( $<10$ bar). Higher surface pressures and high carbon dioxide abundances are generally ruled out, assuming cloud-free atmosphere models. The absence of strong $\ce{CO2}$ features hints at a limited initial volatile inventory or rapid atmospheric escape during the planets' early lifetimes. Taken together, these results motivate a push towards higher precision data, as well as observations of cooler planets that may be more likely to retain atmospheres.  As a goal for future transmission spectroscopy, we suggest a "five scale height challenge," to achieve the precision necessary to detect $\ce{CO2}$ features in nitrogen-rich atmospheres.  Detecting rocky planet atmospheres remains challenging, but with JWST's excellent performance and a continuing investment of telescope time, we are optimistic these uncharted atmospheres will be detected in coming years.

</div>

<div id="div_fig1">

<img src="tmp_2507.00933/./Earth_transmission_best_fit_sensitivity_labeled2.png" alt="Fig2" width="100%"/>

**Figure 2. -** Best-fit model to an empirical transmission spectrum of the cloudless Earth in units of scale height, $H$(adapted from LY2023b).  Different colors indicate the contributions from various molecules.  Despite its relatively low abundance, \ce{CO2} has the most prominent spectral features (at 2.7 and 4.3 {\microns}), making it an ideal molecule to search for in exoplanet atmospheres. The strength of different absorption peaks is sensitive to the underlying atmospheric composition and detailed atmospheric modeling can constrain the abundances of individual absorbing species (which may differ from those of the Earth). This figure neglects the effects of refraction, which effectively blocks transmission below roughly 1.5 scale heights for Earth-Sun analog systems (planets orbiting M-dwarfs are less strongly affected)  (betremieux14) .
 (*fig:earth*)

</div>
<div id="div_fig2">

<img src="tmp_2507.00933/./shoreline.png" alt="Fig1" width="100%"/>

**Figure 1. -** Rocky planets with approved JWST transit and eclipse observations in Cycles 1 -- 4, relative to their escape velocity and lifetime X-ray and UV (XUV) irradiation \citep[adapated from][]{zahnle17}. Cumulative XUV radiation is normalized relative to that of the Earth.  The XUV flux is calculated for a constant system age; a reasonable assumption given that the majority of XUV flux is emitted in the first Gyr of the stars' lifetimes.  The cosmic shoreline is denoted with a gray dashed line, which divides Solar System bodies with substantial atmospheres (below the line) from those without (above). To illustrate the large uncertainties on the possible location (and even existence) of a shoreline around other stars, we shaded the region within an order of magnitude of the relation from zahnle17.  Most planets observed by JWST are situated above the gray dashed line and, thus, are less likely to have atmospheres.
 (*fig:shoreline*)

</div>
<div id="div_fig3">

<img src="tmp_2507.00933/./PNAS-TrSpec-H-v5.png" alt="Fig3" width="100%"/>

**Figure 3. -** Published transmission spectra of rocky exoplanets.  The data have been binned to a uniform resolution and converted to scale heights, $H$, assuming a pure nitrogen atmosphere ($\mu=28$). For clarity, we apply a $40H$ vertical offset between adjacent planets.  The spectra for TRAPPIST-1b and LHS 1140b have been corrected for stellar contamination. The symbol size indicates planet radius and the color indicates planet equilibrium temperature.  Open symbols depict NIRISS data and closed symbols depict NIRSpec or NIRCam data.  The gray regions, which are $5H$ in height, roughly indicate the maximum size of molecular absorption features for a nitrogen-rich atmosphere.  At the current level of precision, none of the spectra are sensitive to features in nitrogen-rich, let alone carbon-dioxide-rich, atmospheres. The blue Earth model is the same as that shown in \autoref{fig:earth}.
 (*fig:transits*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.00933"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

552  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

14  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

7  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
